### 베스트 아이템 페이지 전체의 상품명, 가격, 카테고리 크롤링
1. 베스트 아이템 페이지에서 각 물품의 상세 페이지 클릭
2. 콘텐츠 수집
3. 뒤로가기
#### 1-2-3 반복

In [1]:
from selenium import webdriver as wb # selenium의 webdriver를 사용하기 위한 모듈 (브라우저 할당)
from selenium.webdriver.common.by import By # webdriver 라이브러리에서 웹 요소를 찾기 위한 방법들을 제공하는 모듈
from tqdm import tqdm
import pandas as pd
# import time # 페이지 로딩을 기다리는 데에 사용하는 모듈

In [2]:
driver = wb.Chrome()
driver.get("https://www.gmarket.co.kr/n/best")
driver.maximize_window()

# 전체 상품 리스트
items = []
# 상품 카테고리
categories = []
# 상품 명
names = []
# 가격
prices = []

# 전체 상품 리스트의 길이를 출력하기 위한 호출
items = driver.find_elements(By.CSS_SELECTOR, "a.itemname")

for i in tqdm(range(len(items))) :
    try :
        # 페이지 리로드 시 items 리스트를 다시 가져와야 함!
        # 페이지가 새로 고쳐지면서 이전에 참조했던 요소들이 유효하지 않게 되기 때문
        # Selenium이 특정 요소를 참조할 때, 그 요소는 현재 DOM 구조 내에서의 위치를 기반
        # 요소의 참조는 페이지 로딩 시점의 상태를 기반으로 하므로, DOM이 변경되면 그 참조는 더 이상 유효하지 않음
        items = driver.find_elements(By.CSS_SELECTOR, "a.itemname")
        items[i].click()
        # time.sleep(1) # 슬립
        
        category = driver.find_element(By.CSS_SELECTOR, "li.on > a")
        name = driver.find_element(By.CSS_SELECTOR, "h1.itemtit")
        price = driver.find_element(By.CSS_SELECTOR, "strong.price_real")
        
        categories.append(category.text)
        names.append(name.text)
        prices.append(price.text)
        
        driver.back()
        # time.sleep(1) # 슬립
    except Exception as e :
        print("오류 발생 : ", e)
        continue
driver.quit()

print("크롤링 완료😇")
print(len(categories))
print(len(names))
print(len(prices))

gmarket_dic = {"카테고리" : categories, "상품명" : names, "가격" : prices}
df = pd.DataFrame(gmarket_dic, index = range(1, len(names)+1))
df.index.name = "순위"
df

100%|██████████| 200/200 [15:34<00:00,  4.67s/it]


크롤링 완료😇
200
200
200


,카테고리,상품명,가격
순위,,,
1,메이크업베이스,(유투브 대란핫템) 에뛰드 님프광채 볼류머,"17,000원"
2,라면/면,오늘만 14430원)신라면 5봉 + 너구리 5봉 + 짜파게티 5봉 + 오징어짬뽕 5...,"22,200원"
3,탄산/기능성음료,펩시제로 210ml 30캔 +사이다제로 210ml 30캔,"25,900원"
4,캐주얼원피스,(무료반품/시크세일15%+10%쿠폰)페이지플린 여름신상 베스트아이템 원피스/블라우스...,"19,900원"
5,냉동/즉석식품,햇반 210g 36개 (1박스),"32,730원"
...,...,...,...
196,남아상하복,(롯데백화점)알로봇 쿨 반팔 셋업 5종 택1 133E9-640,"28,760원"
197,기타영양제,다이어트 유산균 비에날씬 450mg x 60캡슐 1개입_MZ,"54,340원"
198,스포츠캡모자,나이키 모자 헤리티지86 퓨추라 레거시91 드라이핏 스우시 볼캡 5종 남성여성 스포...,"14,100원"


In [3]:
df.to_csv("../data/gmarket.csv", encoding = "utf-8")